In [33]:
import geopandas as gpd
import osmnx as ox
import requests
import numpy as np
from PIL import Image
from osgeo import gdal
import os
from pyproj import Transformer
import rasterio


def download_data():
    # Define the target area for which to download the data
    place = 'Japan'

    # Get the boundary of Japan
    # get the first return's bounding box
    url = f"https://nominatim.openstreetmap.org/search?q={place}&format=geojson"
    response = requests.get(url, headers={"User-Agent":"LLM-Find/gladcolor@gmail.com"})
    minx, miny, maxx, maxy = response.json()['features'][0]['bbox']

    # extend the boundary for a point or to small:
    if abs(maxx - minx) < 0.000001:  # note unit is degree
         ext = 0.00005
         maxx = maxx + ext
         minx = minx - ext
         maxy = maxy + ext
         miny = miny - ext 
        
    # Set the zoom level
    z = 4
    n = 2 ** z

    # Calculate the tiling scheme boundaries
    tile_min_col = int((minx + 180.0) / 360.0 * n)
    tile_max_col = int((maxx + 180.0) / 360.0 * n)
    tile_min_row = int((1.0 - np.log(np.tan(np.radians(maxy)) + 1 / np.cos(np.radians(maxy))) / np.pi) / 2.0 * n)
    tile_max_row = int((1.0 - np.log(np.tan(np.radians(miny)) + 1 / np.cos(np.radians(miny))) / np.pi) / 2.0 * n)

    # Create directory to store individual tile images
    save_dir = "tiles"
    os.makedirs(save_dir, exist_ok=True)

    # Download tiles
    for row in range(tile_min_row, tile_max_row + 1):
        for col in range(tile_min_col, tile_max_col + 1):
            url = f"https://services.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{row}/{col}"
            response = requests.get(url)
            tile_path = os.path.join(save_dir, f"{z}-{row}-{col}.jpg")
            with open(tile_path, 'wb') as f:
                f.write(response.content)

    # Stitch the tiles to create a mosaic image
    cols = tile_max_col - tile_min_col + 1
    rows = tile_max_row - tile_min_row + 1
    tile_width, tile_height = Image.open(os.path.join(save_dir, f"{z}-{tile_min_row}-{tile_min_col}.jpg")).size
    mosaic = Image.new('RGB', (cols * tile_width, rows * tile_height))

    for row in range(rows):
        for col in range(cols):
            tile_path = os.path.join(save_dir, f"{z}-{tile_min_row + row}-{tile_min_col + col}.jpg")
            tile = Image.open(tile_path)
            mosaic.paste(tile, (col * tile_width, row * tile_height))

    # Save the mosaic image as a TIFF file
    image_array = np.array(mosaic)

        # Calculate the actual geographic bounds of the mosaic
    def tile_to_lonlat(col, row, zoom):
        n = 2.0 ** zoom
        lon_deg = col / n * 360.0 - 180.0
        lat_rad = np.arctan(np.sinh(np.pi * (1 - 2 * row / n)))
        lat_deg = np.degrees(lat_rad)
        return lon_deg, lat_deg

    actual_min_lon, actual_max_lat = tile_to_lonlat(tile_min_col, tile_min_row, z)
    actual_max_lon, actual_min_lat = tile_to_lonlat(tile_max_col + 1, tile_max_row + 1, z)

       # Convert geographic bounds to Web Mercator
    transformer = Transformer.from_crs("epsg:4326", "epsg:3857", always_xy=True)
    actual_min_x, actual_min_y = transformer.transform(actual_min_lon, actual_min_lat)
    actual_max_x, actual_max_y = transformer.transform(actual_max_lon, actual_max_lat)

    # Set the bounding box for Web Mercator
    actual_bounding_box_mercator = actual_min_x, actual_min_y, actual_max_x, actual_max_y
    print("Actual Bounding Box in Web Mercator:", actual_bounding_box_mercator)
    
    transform = from_bounds(*actual_bounding_box_mercator, width=mosaic.width, height=mosaic.height)

    mosaic_path = "E:/OneDrive_PSU/OneDrive - The Pennsylvania State University/Research_doc/LLM-Find/Downloaded_Data/Japan_image.tif"
    crs = {'init': 'epsg:3857'}

    # Save the image as a GeoTIFF using rasterio
    print(mosaic_path)
    with rasterio.open(
        mosaic_path,
        'w',
        driver='GTiff',
        height=image_array.shape[0],
        width=image_array.shape[1],
        count=image_array.shape[2],  # Number of channels (e.g., 3 for RGB)
        dtype=image_array.dtype,
        crs=crs,
        transform=transform
    ) as dst:
        # Write each channel separately
        for i in range(1, image_array.shape[2] + 1):
            print(i)
            dst.write(image_array[:, :, i - 1], i)
 
    # Clean up the individual tile images (optional)
    for file in os.listdir(save_dir):
        os.remove(os.path.join(save_dir, file))
    os.rmdir(save_dir)

download_data()

Actual Bounding Box in Web Mercator: (12523442.714243276, 0.0, 17532819.799940586, 7514065.628545966)
E:/OneDrive_PSU/OneDrive - The Pennsylvania State University/Research_doc/LLM-Find/Downloaded_Data/Japan_image.tif
1
2
3


In [ ]:
import requests
import csv
import json

def download_data():
    # Define the endpoint and related configurations
    base_url = "https://api.census.gov/data/2021/acs/acs5"
    api_key = "{US_Census_key}"
    dataset_year = "2021"
    dataset_source = f"ACS {dataset_year}"

    # Variables we need to fetch
    variables = [
        # B02001_001E: Total population
        "B02001_001E",
        # B02001_002E: White alone
        "B02001_002E",
        # B02001_003E: Black or African American alone
        "B02001_003E",
        # B02001_004E: American Indian and Alaska Native alone
        "B02001_004E",
        # B02001_005E: Asian alone
        "B02001_005E",
        # B02001_006E: Native Hawaiian and Other Pacific Islander alone
        "B02001_006E",
        # B02001_007E: Some other race alone
        "B02001_007E",
        # B02001_008E: Two or more races
        "B02001_008E",
    ]

    # Fetch the variable labels
    response = requests.get(f"{base_url}/variables.json")
    print(f"{base_url}/variables.json")
    
    response.raise_for_status()
    # print(response.text[:10000])
    variables_metadata = response.json()

    # Helper function to get variable labels
    def get_variable_label(var_name):
        label = variables_metadata['variables'][var_name]['label']
        return label.replace("Estimate!!", "").strip()

    # Construct the URL for the data request
    get_vars = ",".join(variables)
    url = f"{base_url}?get={get_vars}&for=county:*&in=state:*&key={api_key}"

    # Download data from Census API
    response = requests.get(url)
    response.raise_for_status()
    data = response.json()

    # Prepare the CSV file for writing
    csv_path = "E:\\OneDrive_PSU\\OneDrive - The Pennsylvania State University\\Research_doc\\LLM-Find\\Python_code\\Downloaded_Data\\Census_US_county_population_by_race.csv"

    # Create header with variable labels
    header = [f"{var}:{get_variable_label(var)}" for var in variables] + ["state_fips", "county_fips", "year", "source"]
    rows = data[1:]  # Skip the header row provided by API
    for row in rows:
        row.extend([dataset_year, dataset_source])

    # Write to CSV file
    with open(csv_path, mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(header)
        writer.writerows(rows)

# Execute the function
download_data()

In [46]:
import requests
import csv
import json

def download_data():
    # Define the endpoint and related configurations
    base_url = "https://api.census.gov/data/2022/acs/acs5"
    api_key = "xxxx"
    dataset_year = "2021"
    dataset_source = f"ACS {dataset_year}"

    # Variables we need to fetch
    variables = [
        # B02001_001E:Total population
        "B02001_001E",
        # B02001_002E:White alone
        "B02001_002E",
        # B02001_003E:Black or African American alone
        "B02001_003E",
        # B02001_004E:American Indian and Alaska Native alone
        "B02001_004E",
        # B02001_005E:Asian alone
        "B02001_005E",
        # B02001_006E:Native Hawaiian and Other Pacific Islander alone
        "B02001_006E",
        # B02001_007E:Some other race alone
        "B02001_007E",
        # B02001_008E:Two or more races
        "B02001_008E",
    ]

    # Fetch the variable labels
    response = requests.get(f"{base_url}/variables.json")
    variables_metadata = response.json()

    # Helper function to get variable labels
    def get_variable_label(var_name):
        label = variables_metadata['variables'][var_name]['label']
        return label.replace("Estimate!!", "").strip()

    # Construct the URL for the data request
    get_vars = ",".join(variables)
    url = f"{base_url}?get={get_vars}&for=block%20group:*&in=state:45 county:079&key={api_key}"

    # Download data from Census API
    response = requests.get(url)
    response.raise_for_status()
    data = response.json()

    # Prepare the CSV file for writing
    csv_path = "Census_SC_Richland_race_population.csv"

    # Create header with variable labels
    header = [f"{var}:{get_variable_label(var)}" for var in variables] + ["state_fips", "county_fips", "tract_fips", "block_group_fips", "year", "source"]
    rows = data[1:]  # Skip the header row provided by API
    for row in rows:
        row.extend([dataset_year, dataset_source])

    # Write to CSV file
    with open(csv_path, mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(header)
        writer.writerows(rows)

# Execute the function
download_data()

JSONDecodeError: Expecting value: line 2 column 1 (char 1)

In [20]:
from PIL import Image
import numpy as np
import rasterio
from rasterio.transform import from_bounds

# Load the image using PIL
pil_image = Image.open(r"E:\OneDrive_PSU\OneDrive - The Pennsylvania State University\Research_doc\LLM-Find\Python_code\Downloaded_Data\Everest_DOM.tif")

# Convert the PIL image to a NumPy array
image_array = np.array(pil_image)

# Define the bounding box (left, bottom, right, top)
# Example coordinates (in desired CRS units, e.g., meters or degrees)
bounding_box = 86.73, 27.82, 87.13, 28.17# from_bounds(west, south, east, north, width, height)

# Define the transform for the GeoTIFF based on the bounding box
transform = from_bounds(*bounding_box, width=pil_image.width, height=pil_image.height)

# Define the CRS (coordinate reference system) for the output GeoTIFF
# Example using WGS 84 (EPSG:4326), but change as needed for your data
crs = {'init': 'epsg:4326'}

# Save the image as a GeoTIFF using rasterio
with rasterio.open(
    r'E:\OneDrive_PSU\OneDrive - The Pennsylvania State University\Research_doc\LLM-Find\Python_code\Downloaded_Data\output_image.tif',
    'w',
    driver='GTiff',
    height=image_array.shape[0],
    width=image_array.shape[1],
    count=image_array.shape[2],  # Number of channels (e.g., 3 for RGB)
    dtype=image_array.dtype,
    crs=crs,
    transform=transform
) as dst:
    # Write each channel separately
    for i in range(1, image_array.shape[2] + 1):
        print(i)
        dst.write(image_array[:, :, i - 1], i)

1
2
3


In [13]:
import toml

handbook_file = r'D:\OneDrive_PSU\OneDrive - The Pennsylvania State University\Research_doc\LLM-Find\Python_code\Handbooks\OpenStreetMap.toml'

handbook = toml.load(handbook_file)
print(handbook['code_example'])

## The following code is to download the railway network in Wuhan, Hubei, China.
# Import necessary libraries
import geopandas as gpd
import requests
import json
import osmnx as ox

def download_data():
    # Define the area for Wuhan, Hubei, China
    place_name = "Wuhan, Hubei, China"

    # Get the bounding box of Wuhan using OSMnx
    gdf = ox.geocode_to_gdf(place_name)
    west, south, east, north = gdf.unary_union.bounds

    # Define Overpass API query to get railway network in the bounding box
    overpass_url = "https://overpass-api.de/api/interpreter"
    overpass_query = f """    [out:json];
    (
        way["railway"]({south},{west},{north},{east});
        relation["railway"]({south},{west},{north},{east});
    );
    out geom;
      """    # Send request to Overpass API
    response = requests.get(overpass_url, params={'data': overpass_query})
    response.raise_for_status()  # Automatically raises an error for bad status codes

    # Parse the JSON response
    data = r

In [30]:
# pip install toml
os.path.join('Keys', 'config.ini')

'Keys\\config.ini'

In [54]:
import configparser
import LLM_Find_Codebase as codebase
import os
config = configparser.ConfigParser()
config.read(os.path.join('Keys', 'US_Census_demography.keys'))
# config.read('Keys/OpenWeather.keys')
# config.read('Keys/config.ini')
config['API_Key'].keys()

KeysView(<Section: API_Key>)

In [55]:
config['API_Key'].keys()

KeysView(<Section: API_Key>)

In [57]:
for key in config['API_Key'].keys():
    value = print(key)

us_census_demography


In [ ]:
keys_dict = handbook.load_keys(source_ID='US_Census_demography', keys_dir='Keys')
print(keys_dict)
